# Cohere : Fine tuned model testing

#### Intro to fine-tuning
https://docs.cohere.com/docs/fine-tuning

#### Hyperparameters
https://github.com/cohere-ai/cohere-aws/blob/main/notebooks/sagemaker/Run%20command%20R%20finetuning.ipynb



#### Google Colab

If you are running the code in Google colab, install the packages by uncommenting/running the cell below

In [ ]:
# !pip install python-dotenv

## 1. Setup environment variables

This will read the local enviroment file. You MUST change the location of the **env** file

In [1]:
from dotenv import load_dotenv
import os
import sys
import warnings

# Setting path so we can access the utils folder
sys.path.append('../')
sys.path.append('./')

from IPython.display import Markdown, JSON

warnings.filterwarnings("ignore")

# Load the file that contains the API keys - CHANGE THIS
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')

True

## 2. Utility class
Invocation of the fine-tuned model

In [3]:
# Use the Cohere REST endpoint for invoking the model

import requests
import getpass
import os

class cohere_rest_client:
    # Holds the model to be invoked
    model = ""

    # Holds api URL
    api_url = ""
    
    # Holds the API token
    COHERE_API_KEY=""

    # Initialize
    def  __init__(self, model, api_url="https://api.cohere.com/v2/classify", api_key=None):
        self.api_url = api_url
        self.model = model

        # Check if API token is provided, if not then check env, if not ask for it
        if not api_key:
            self.COHERE_API_KEY=os.getenv('COHERE_API_KEY')
            if not self.COHERE_API_KEY:
                print("Provide the COHERE_API_KEY:")
                self.COHERE_API_KEY=getpass.getpass()
        else:
            self.COHERE_API_KEY=api_key

    # Returns the model's API URL
    def get_model_url(self):
        return self.api_url

    # Invoke function
    def invoke(self, inputs, examples = None, parameters = {}, options={}):

        # Setup header with API token
        headers = {
            "Authorization": f"Bearer {self.COHERE_API_KEY}",
            "Content-Type": "application/json"
        }
        

        # Create the payload JSON
        payload =  {
            "model": self.model,
            "inputs": inputs,
            "parameters": parameters
        }

        

        if  examples:
            payload['examples'] = examples

        print(payload)

        # print(payload)

        # Post to the model URL
        response = requests.post(self.api_url, headers=headers, json=payload)

        # Check for errors
        if response.status_code != 200:
            # error !!
            return {
                "status_code": response.status_code,
                "reason" : response.reason,
                "error": True
            }
        else:
            # Return response as JSON
            return response.json()


## 3. Unit test the utility

In [4]:
# Test if the fine tuned model is responding
cohere_api_url = "https://api.cohere.com/v2/classify"

# Copy from Dashboard & paste it here [Model MUST be in Ready state]
model = "862c41b7-55c7-4625-98ad-426ca1d295b3-ft"

api_client = cohere_rest_client(model=model, api_url=cohere_api_url)

In [7]:
from IPython.display import Markdown, JSON

inputs = [
    "i like you"
]

response = api_client.invoke(inputs)

JSON(response)

{'model': '862c41b7-55c7-4625-98ad-426ca1d295b3-ft', 'inputs': ['i like you'], 'parameters': {}}


<IPython.core.display.JSON object>

## 4. Evaluate model

* Read the test dataset
* For each line in the dataset call inference
* Check accuracy by comparing predicted with predicted

### 4.1 Read test dataset and prepare inputs

On Google Colab?
Uncomment the following cell and run it. It will download the test data file.

In [8]:
# !mkdir  -p ./data/toxicity-classifier/
# !curl  https://raw.githubusercontent.com/acloudfan/gen-ai-app-dev/refs/heads/main/Fine-Tuning/data/toxicity-classifier/multi_label_comment_classification_test_cohere.jsonl -o ./data/toxicity-classifier/multi_label_comment_classification_test_cohere.jsonl

In [9]:
import json

with open('./data/toxicity-classifier/multi_label_comment_classification_test_cohere.jsonl', 'r') as json_file:
    json_list = list(json_file)

# Prepare the inputs array that we will send to 
inputs = []
labels = []
for json_str in json_list: 
    result = json.loads(json_str)
    inputs.append(result['text'])
    labels.append(result['label'])
    

### 4.2 Invoke model 

In [10]:
responses = api_client.invoke(inputs)

# JSON(response)

{'model': '862c41b7-55c7-4625-98ad-426ca1d295b3-ft', 'inputs': [':::::::::Moi? Ego? I am mortified that you could say such a thing - poor old Mona I always thiught she was a miserable looking woman - probably hormonal.', '" \n\n So, on the tenth anniversary of 9/11, ""New York Times"" ""columnist"" Paul Krugman delivered the odious piece of dribble ever on the attack  well, at least since the last time he wrote about it. The headline is, quote, ""The Years of Shame,"" end quote. And no, it\'s not directed at radical Islam. Instead, he targets America and the, quote, ""fake heroes."" Krugman claims that the atrocity has been hijacked. The sick word given that he\'s not actually talking about the real hijackers who killed some 3,000 Americans. No, he\'s referring to people like George Bush and Rudy Giuliani who he thinks capitalized on the horrible crimes. But the worst part, at the end of this column, the creep writes, ""I\'m not going to allow comments on the post for obvious reasons."

### 4.3 Compare predicted vs original

In [11]:
correct = 0
in_correct = 0

for i, response in enumerate(responses["classifications"]):
    # print(response["predictions"])

    # Using the set as order of elements don't matter
    if set(response['predictions']) == set(labels[i]):
        correct = correct + 1
    else:
        in_correct = in_correct + 1
        # print(response['predictions'], "   ", labels[i])

In [12]:
print("Correct:   ", correct)
print("Incorrect: ", in_correct)

print("Accuracy:  ", int((correct/(correct+in_correct))*100),"%")

Correct:    17
Incorrect:  18
Accuracy:   48 %
